In [137]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [138]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [139]:
# Global Parameters
R = 0.082057 # In atm*L/mol/K
N = 1.0 # In mols
pts = 100 # Number of points for each process

In [140]:
def get_intersect(vi,pi,vf,pf,process1,process2):
    if process1 == 'v' and process2 == 'p':
        vc = vi
        pc = pf
    if process1 == 'p' and process2 == 'v':
        vc = vf
        pc = pi
    if process1 == 'v' and process2 == 'T':
        vc = vi
        pc = vf/vc*pf 
    if process1 == 'T' and process2 == 'v':
        vc = vf
        pc = vi/vc*pi
    if process1 == 'p' and process2 == 'T':
        pc = pi
        vc = pf/pc*vf
    if process1 == 'T' and process2 == 'p':
        pc = pf
        vc = pi/pc*vi
    return vc,pc

In [141]:
def get_isotherm(v,pa,pb):
    T = pa*v[0]/N/R
    p = N*R*T/v
    return p

In [144]:
def get_p_values(v,pa,pb,process):
    if process == 'T':
        p = get_isotherm(v,pa,pb)
    if process == 'p':
        p = np.linspace(pa,pb,pts)
    if process == 'v':
        p = np.linspace(pa,pb,pts)
    return p

In [145]:
def get_process_values(pstr):
    
    if pstr == 'isokoro-isobaro':
        process1, process2 = 'v','p'
    if pstr == 'isobaro-isokoro':
        process1, process2 = 'p','v'
    if pstr == 'isokoro-isotermo':
        process1, process2 = 'v','T'
    if pstr == 'isotermo-isokoro':
        process1, process2 = 'T','v'
    if pstr == 'isobaro-isotermo':
        process1, process2 = 'p','T'
    if pstr == 'isotermo-isobaro':
        process1, process2 = 'T','p'
    return process1, process2

In [146]:
def calculate_work(v1,p1,v2,p2):
    W= sum(p1)*(v1[pts-1] - v1[0])/pts + sum(p2)*(v2[pts-1] - v2[0])/pts
    return W

In [147]:
def calculate_energy(vi,pi,vf,pf):
    dU = 1.5*(vf*pf - vi*pi)
    return dU

In [148]:
def update_figure(change):
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    pf = pf_slider.value
    process_string = process_sel.value

    process1,process2 = get_process_values(process_string)
    

    vc,pc = get_intersect(vi,pi,vf,pf,process1,process2)
    v1 = np.linspace(vi,vc,pts)
    v2 = np.linspace(vc,vf,pts)
    p1 = get_p_values(v1,pi,pc,process1)
    p2 = get_p_values(v2,pc,pf,process2)
    
    work = calculate_work(v1,p1,v2,p2)
    energy = calculate_energy(vi,pi,vf,pf)
    
    work_value_text.value = '%.2f' % work
    energy_value_text.value = '%.2f' % energy
    
    heat_value_text.value = '%.2f' % (energy - work)
        
    x_values = []
    y_values = []
    for i in range(pts):
        x_values.append(v1[i])
        y_values.append(p1[i])
    for i in range(pts):
        x_values.append(v2[i])
        y_values.append(p2[i])
        
    curve.x = x_values
    curve.y = y_values
    InitialPoint.x = [vi]
    InitialPoint.y = [pi]
    FinalPoint.x = [vf]
    FinalPoint.y = [pf]

In [149]:
# Initial values
vi = 1.0
pi = 1.0
Ti = vi*pi/N/R
process1 = 'v'
process2 = 'p'

# Final values
vf = 2.0
pf = 0.8
Tf = vf*pf/N/R

# Get curves
vc,pc = get_intersect(vi,pi,vf,pf,process1,process2)
v1 = np.linspace(vi,vc,pts)
v2 = np.linspace(vc,vf,pts)
p1 = get_p_values(v1,pi,pc,process1)
p2 = get_p_values(v2,pc,pf,process2)


In [150]:
########################
###CREATE THE FIGURES###
########################

fig_122_001 = bq.Figure(title='Prozesu konposatua p-v',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
    )

scale_x = bqs.LinearScale(min = 0.0, max = 10.0)
scale_y = bqs.LinearScale(min = 0.0, max = 10.0)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_122_001.axes = [axis_x,axis_y]


########################
####CREATE THE MARKS####
########################

marks = []

x_values = []
y_values = []
for i in range(pts):
    x_values.append(v1[i])
    y_values.append(p1[i])
for i in range(pts):
    x_values.append(v2[i])
    y_values.append(p2[i])

color = ['#0079c4']

curve = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y},
                colors = color,
                fill_colors = ['green'],
                fill_opacities = [0.35],
                fill = 'bottom',
)

marks.append(curve)

InitialPoint = bqm.Scatter(
    name = 'Initial Point',
    x = [vi],
    y = [pi],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=['tracer']
)      

marks.append(InitialPoint)

FinalPoint = bqm.Scatter(
    name = 'Final Point',
    x = [vf],
    y = [pf],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=['tracer']
)      

marks.append(FinalPoint)

fig_122_001.marks = marks


########################
######  WIDGETS  #######
########################

vi_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='$v_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

vf_slider = widgets.FloatSlider(
    value=2.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='$v_f$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

pi_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='$p_i$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

pf_slider = widgets.FloatSlider(
    value=2.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='$p_f$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

process_sel = widgets.Dropdown(
    options=['isokoro-isobaro', 'isobaro-isokoro', 'isokoro-isotermo', 'isotermo-isokoro', 'isobaro-isotermo', 'isotermo-isokoro'],
    value='isokoro-isobaro',
    description='Prozesua',
    disabled=False,
)

vi_slider.observe(update_figure, 'value')
pi_slider.observe(update_figure, 'value')
vf_slider.observe(update_figure, 'value')
pf_slider.observe(update_figure, 'value')
process_sel.observe(update_figure, 'value')


########################
######  OUTPUTS  #######
########################

work_value_text = widgets.HTML(
    value="",
)

energy_value_text = widgets.HTML(
    value="",
) 

heat_value_text = widgets.HTML(
    value="",
) 




########################
######  LAYOUT  ########
########################

## Left Block ##
left_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%'))
left_block_122_000.children = [vi_slider,pi_slider,vf_slider,pf_slider,process_sel]

## Figure Block ##
figure_block_122_000 = widgets.VBox([fig_122_001], layout=widgets.Layout(width='60%'))
figure_block_122_000.children = [fig_122_001]

## Right Block ##
right_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%'))
right_block_122_000.children = [
                       widgets.HBox([widgets.Label(value="$W:$"),work_value_text,]),
                       widgets.HBox([widgets.Label(value="$\Delta U:$"),energy_value_text,]),
                       widgets.HBox([widgets.Label(value="$Q:$"),heat_value_text,])
                      ]


## Main Block ##

main_block_122_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_122_000.children = [left_block_122_000, figure_block_122_000,right_block_122_000]

main_block_122_000